# Sensitivity analysis with uncertainty

This notebook illustrates how to do sensitivity analysis with uncertainty.


There are 3 uncertainty strategy defined in this library:
1) `uniformly`: This strategy add the same type/value of the uncertainty to a matrix, but you can add uncertainty for all matricies or only one/two of the matrices.
2) `columnwise`: This strategy add the same type/value of the uncertainty to the same column of a matrix, different column can have different type/value of uncertainty. The uncertainty first save into metadata and then add to the uncertainty array accordingly. To use this stragety, your uncertainty input should be defined in the file or save it directly to metadata of this library.
3) `itemwise`: This strategy add different type/value of the uncertainty different element in the matrix. To use this stragety, your uncertainty input should be defined in the file or save it directly to metadata of this library.

Note: for strategy 2) and 3), only technosphere and biosphere matrices are supported.

### Preparation

1. Direct to the correct Brightway project and ensure the background database is loaded.

In [1]:
import bw2data as bd

bd.projects.set_current("advlca25")
bd.databases

Databases dictionary with 4 object(s):
	ALIGNED-biob-prod-dummy
	Pyro Phenol
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential

2. Importing required libraries

In [2]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_bamboo.background_importer import *
from bw_bamboo.foreground_importer import *
from bw_bamboo.datapackage_builder import *
from bw_bamboo.uncertainty_handler import *
from bw_bamboo.lca_wrapper import *
from bw_bamboo.uncertainty_importer import *

3. Define some constants

In [3]:
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - fossil - air",
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

METHOD = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')

# BACKGROUND DATABASE FILE PATH
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")

# FOREGROUND DATABASE FILE PATH
FROEGROUND_1 = os.path.join(os.getcwd(), "data/foreground_system_1.csv")
FROEGROUND_2 = os.path.join(os.getcwd(), "data/foreground_system_2.csv")

# CHARACTERIZATION FACTOR MAPPING FILE PATH
CF_MAPPING_FILE = os.path.join(os.getcwd(), "data/cf_mapping_file.csv")

### Step 1: Get the background database matrices

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.build_tech_matrix(raw_tech)

print(f"The shape of technosphere matrix is: {tech_matrix.shape}")

The shape of technosphere matrix is: (76, 76)


2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.build_bio_matrix(bio_df, GHG)
print(f"The shape of biosphere matrix is: {bio_matrix.shape}")

The shape of biosphere matrix is: (18, 76)


3. Get characterization factor matrix

You can choose to search for CFs from the biosphere database, or you can put CFs directly into the file.

In [6]:
# get characterization factor matrix from code.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG, "ecoinvent-3.11-biosphere", METHOD, source="code")

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 2.850000e+01
 2.970000e+01 2.970000e+01 2.970000e+01 2.970000e+01 2.970000e+01
 2.970000e+01 2.970000e+01 2.970000e+01 2.970000e+01 2.970000e+01
 2.648000e+02 2.648000e+02 2.350682e+04]


In [7]:
# get characterization factor matrix from the file directly.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG)  # By default, source="cf", so you can omit some parameters.

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00e+00 1.00e+00 1.00e+00 1.00e+00 2.70e+01 2.98e+01 2.98e+01 2.98e+01
 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.73e+02
 2.73e+02 2.52e+04]


### Step 2: Import the foreground database

4. Check out the foreground data

In [8]:
fg_tech_df = pd.read_table(FROEGROUND_2, sep=',')
fg_tech_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,GSD,Exchange negative
0,column_1,EU28-Agriculture-Forestry-Fishing,technosphere,0.000000,0,NaN,NaN
1,column_1,EU28-Energy,technosphere,NaN,0,NaN,NaN
2,column_1,EU28-Natural gas and services related to natur...,technosphere,NaN,0,NaN,NaN
3,column_1,EU28-Industry,technosphere,0.000001,2,3.330005,False
4,column_1,EU28-Motor Gasoline,technosphere,-0.000016,2,1.540000,True


5. Get activities in foreground system.

In [9]:
fg_activities = get_fg_activities(FROEGROUND_2, ",")
fg_activities

['column_1', 'column_2']

6. Get activities in background system.

In [10]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities[:5] # only list the first five activities here

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline']

7. Get foreground system matrices

In [11]:
fg_importer = ForegroundImporter()

fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_tech_df, GHG, fg_activities, bg_activities)

8. Concatenate foreground system with background system

In [12]:
full_tech_matrix, full_bio_matrix = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg, fgfg, bgfg, bifg)

# to compare the matrix before and after concatenate with foreground system
print(f"Technosphere shape change from {tech_matrix.shape} to {full_tech_matrix.shape}.")
print(f"Biosphere shape change from {bio_matrix.shape} to {full_bio_matrix.shape}.")

Technosphere shape change from (76, 76) to (78, 78).
Biosphere shape change from (18, 76) to (18, 78).


### Step 3: Save uncertainty into metadata

9. Get activities for both foreground system and background system.

In [13]:
activities = fg_activities + bg_activities  # note: we always want foreground activities to be in front.
activities[:5]  # only print the first 5 activities

['column_1',
 'column_2',
 'EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying']

10. Store uncertainty information in library metadata

In [14]:
uncertainty_importer = UncertaintyImporter(FROEGROUND_2, ",")
uncertainty_importer.update_metadata_uncertainty(activities, "itemwise")  # here the uncertainty strategy is set to "itemwise"
uncertainty_importer.metadata

{0: {'Activity name': 'column_1',
  'Activity uncertainty type': 2,
  'Exchange uncertainty amount': [1.111,
   0.0,
   0.0,
   0.0,
   0.0,
   3.33000452,
   1.54,
   1.54,
   0.0,
   1.54,
   1.54,
   4.09770687,
   0.0,
   1.59,
   0.0,
   0.0,
   2.014014413,
   0.0,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   0.0,
   2.674794737,
   2.674794737,
   0.0,
   2.674794737,
   2.288684673,
   0.0,
   0.0,
   0.0,
   2.238117654,
   2.699910682,
   0.0,
   1.56,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  'Exchange negative': [False,
   False,
   True,
   True,
   True,
   False,
   True,
   True,
   True,
   False,
   True,
   False,


### Step 4: Build datapackage

11. Prepare datapackage matrix data

In [15]:
dp_builder = DatapackageBuilder()
(full_tech_data, full_tech_indices, full_tech_flip), (full_bio_data, full_bio_indices), (cf_data, cf_indices) = dp_builder.prepare_dp_matrix(full_tech_matrix, full_bio_matrix, cf_matrix)

12. Build uncertainty arrays for datapackage

In [16]:
uncertainty_handler = UncertaintyHandler()
full_tech_uncertainty_array = uncertainty_handler.add_nonuniform_uncertainty(full_tech_data, full_tech_indices, full_tech_flip, "itemwise", fg_num=2, fg_strategy="itemwise")
full_bio_uncertainty_array = uncertainty_handler.add_nonuniform_uncertainty(full_bio_data, full_bio_indices, None, "itemwise", fg_num=2, fg_strategy="itemwise")

13. Check the shape of the uncertainty array.  
- As long as the data shape is identical to the uncertainty shape, it's correct.  
- Note: the shape is not identical to the matrix shape (76 * 76), (20 * 76) or (20), because a sparse matrix does not store zero data.

In [17]:
print(f"The length of technosphere and biosphere array: {full_tech_data.shape, full_bio_data.shape}")
print(f"The length of technosphere and biosphere uncertainty array: {full_tech_uncertainty_array.shape, full_bio_uncertainty_array.shape}")

The length of technosphere and biosphere array: ((5556,), (1291,))
The length of technosphere and biosphere uncertainty array: ((5556,), (1291,))


14. Check the uncertainty data

In [18]:
print(f"Information store in uncertainty array: \n {[dtype[0]for dtype in bwp.UNCERTAINTY_DTYPE]} \n")
print(f"Examples of uncertainty data required in datapackage: \n {full_tech_uncertainty_array[:10]} \n")

Information store in uncertainty array: 
 ['uncertainty_type', 'loc', 'scale', 'shape', 'minimum', 'maximum', 'negative'] 

Examples of uncertainty data required in datapackage: 
 [(2, 0.        , 0.10526051, nan, nan, nan, False)
 (2, 0.        , 0.10526051, nan, nan, nan, False)
 (0, 0.7363383 ,        nan, nan, nan, nan, False)
 (0, 0.00132513,        nan, nan, nan, nan, False)
 (0, 0.00250369,        nan, nan, nan, nan, False)
 (0, 0.00457712,        nan, nan, nan, nan, False)
 (0, 0.00210603,        nan, nan, nan, nan, False)
 (0, 0.00186496,        nan, nan, nan, nan, False)
 (0, 0.00194427,        nan, nan, nan, nan, False)
 (0, 0.00127726,        nan, nan, nan, nan, False)] 



15. Build datapackage

In [19]:
datapackage_data = [(full_tech_data, full_tech_indices, full_tech_flip), (full_bio_data, full_bio_indices), (cf_data, cf_indices)]
uncertainty = [full_tech_uncertainty_array, full_bio_uncertainty_array, None]

dp = dp_builder.prepare_datapackage(datapackage_data, uncertainty)
dp.data

[array([( 0,  0), ( 1,  1), ( 2,  2), ..., (77, 75), (77, 76), (77, 77)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([1.00000000e+00, 1.00000000e+00, 7.36338343e-01, ...,
        6.24171469e-03, 9.40994608e-04, 9.95178298e-01]),
 array([(2, 0.0000000e+00, 0.10526051, nan, nan, nan, False),
        (2, 0.0000000e+00, 0.10526051, nan, nan, nan, False),
        (0, 7.3633832e-01,        nan, nan, nan, nan, False), ...,
        (0, 6.2417146e-03,        nan, nan, nan, nan, False),
        (0, 9.4099459e-04,        nan, nan, nan, nan, False),
        (0, 9.9517828e-01,        nan, nan, nan, nan, False)],
       dtype=[('uncertainty_type', 'u1'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')]),
 array([False, False, False, ...,  True,  True, False]),
 array([(78,  2), (78,  3), (78,  4), ..., (95, 75), (95, 76), (95, 77)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([6.92719535e+04, 2.24902521e+05, 1.58717

### Step 5: Run LCA

In [20]:
mysector = 0

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[dp],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")

Brightway calculated lca score: (-98.51135421031272, 'EU28-Agriculture-Forestry-Fishing')


In [21]:
mysector = 0

for i in range(10):
    lca = bc.LCA(
                demand={mysector : 1},
                data_objs=[dp],
                use_distributions=True,
            )
    lca.lci()
    lca.lcia()

    print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")

Brightway calculated lca score: (245.63948543537182, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (375.49179952458564, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (319.6896997238435, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (254.24336001952955, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (171.0655953456148, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (248.09929990616004, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (132.30236480084608, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (257.360960910757, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (161.74109286916524, 'EU28-Agriculture-Forestry-Fishing')
Brightway calculated lca score: (166.9427877259631, 'EU28-Agriculture-Forestry-Fishing')


### Step 6: Compare to manually LCA

In [22]:
np.fill_diagonal(full_tech_matrix, -full_tech_matrix.diagonal())
full_tech_matrix_2 = -full_tech_matrix

rca = LCAWrapper()
rca.manual_lca(full_tech_matrix_2, full_bio_matrix, cf_matrix, mysector)

-98.51135421031277